In [ ]:
!nvidia-smi

Fri Mar  8 11:02:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
#@title Train with Kohya's Stable Diffusion Trainers
%cd /content

# from google.colab import drive
# drive.mount('/content/drive')

!mkdir -p /content/drive

!pip install dadaptation diffusers[torch] easygui einops fairscale ftfy gradio huggingface-hub
!pip install lion-pytorch lycoris_lora open-clip-torch prodigyopt pytorch-lightning safetensors timm
!pip install tk transformers voluptuous wandb xformers omegaconf

/content
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 29.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py)

In [1]:
%cd /content
!git clone https://github.com/TimDettmers/bitsandbytes
%cd /content/bitsandbytes
!python setup.py install

/content
Cloning into 'bitsandbytes'...
remote: Enumerating objects: 5741, done.
remote: Counting objects: 100% (1531/1531), done.
remote: Compressing objects: 100% (232/232), done.
remote: Total 5741 (delta 1398), reused 1351 (delta 1290), pack-reused 4210
Receiving objects: 100% (5741/5741), 1.73 MiB | 11.92 MiB/s, done.
Resolving deltas: 100% (3875/3875), done.
/content/bitsandbytes
libs: []
/usr/local/lib/python3.10/dist-packages/setuptools/dist.py:519: InformationOnly: Normalizing '0.44.0.dev' to '0.44.0.dev0'
  self.metadata.version = self._normalize_version(
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssl

In [2]:
%cd /content
!git clone https://github.com/bmaltais/kohya_ss
%cd /content/kohya_ss

/content
Cloning into 'kohya_ss'...
remote: Enumerating objects: 11785, done.
remote: Counting objects: 100% (1369/1369), done.
remote: Compressing objects: 100% (498/498), done.
remote: Total 11785 (delta 915), reused 1260 (delta 843), pack-reused 10416
Receiving objects: 100% (11785/11785), 21.84 MiB | 28.23 MiB/s, done.
Resolving deltas: 100% (8081/8081), done.
/content/kohya_ss


In [ ]:
!python kohya_gui.py --share --headless

11:22:58-210144 INFO     headless: True                                                             
11:22:58-214195 INFO     Load CSS...                                                                
Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://0ea53eafa7e8347d6a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
#@title Convert Safetensors to Diffusers
from_safetensors_url = '' #@param {type:"string"}
!wget -q https://raw.githubusercontent.com/huggingface/diffusers/v0.17.1/scripts/convert_original_stable_diffusion_to_diffusers.py
!wget {from_safetensors_url} -O /content/model.safetensors
!python3 convert_original_stable_diffusion_to_diffusers.py --half --from_safetensors --checkpoint_path model.safetensors --dump_path /content/model

In [ ]:
#@title Push to HF.co

import os
from huggingface_hub import create_repo, upload_folder

hf_token = 'HF_WRITE_TOKEN' #@param {type:"string"}
repo_id = 'username/reponame' #@param {type:"string"}
commit_message = '\u2764' #@param {type:"string"}
create_repo(repo_id, private=True, token=hf_token)
model_path = '/content/train/model' #@param {type:"string"}
upload_folder(folder_path=f'{model_path}', path_in_repo='', repo_id=repo_id, commit_message=commit_message, token=hf_token)

In [ ]:
#@title Push to DagsHub.com

!pip -q install dagshub
from dagshub.upload import Repo, create_repo

repo_id = 'reponame' #@param {type:"string"}
org_name = 'orgname' #@param {type:"string"}
commit_message = '\u2764' #@param {type:"string"}
create_repo(f"{repo_id}", org_name=f"{org_name}")
repo = Repo(f"{org_name}", f"{repo_id}")
model_path = '/content/train/model' #@param {type:"string"}
repo.upload("/content/models", remote_path="data", commit_message=f"{commit_message}", force=True)